# Jet Events: Fetching Data from CDAWeb
The purpose of this notebook is to compute dynamic pressure and compile satellite parameters for a list of jet events that has already been generated based on  measurements from MMS satellites. The data were downloaded locally from the sources listed below before running the code in this notebook.

The MMS data set can be found at: https://osf.io/hz2n6/
Specifically, we want this file: https://osf.io/download/qyvzp/

You can reference this data set as follows: Plaschke, F., Jernej, M., Hietala, H., & Vuorinen, L. (2020, October 27). On the alignment of velocity and magnetic fields within magnetosheath jets: Data sets. Retrieved from osf.io/hz2n6.

The description of the identification processes of the MMS data sets is published in: Plaschke, F., Jernej, M., Hietala, H., and Vuorinen, L.: On the alignment of velocity and magnetic fields within magnetosheath jets, Ann. Geophys., 38, 287–296, https://doi.org/10.5194/angeo-38-287-2020, 2020.


In [1]:
# Importing packages
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np

# For pulling data from CDAweb:
from ai import cdas
import datetime
import pyspedas

import pickle

## MMS Events (2015 - 2017)

The columns contain:
*    column 1: jet number
*    column 2: observing spacecraft (1: MMS 1, ..., 4: MMS 4)
*    column 3: start of identified jet interval in UT
*    column 4: time of maximum dynamic pressure ratio in UT
*    column 5: end of identified jet interval in UT

In the final version of the list, the column is indexed by "Max" (col 4 in original), and the other columns are named 'Jet Number', 'Ref Spacecraft', 'Start', and 'End'.

From CDAWeb, we add the following columns: 
*    **SM_LAT, SM_LON**: Latitude and longitude in GSM coordinates
*    **SM_X, SM_Y, SM_Z**: Cartesian position in GSM coordinates
*    **GEO_X_1, GEO_Y_1, GEO_Z_1**: Cartesian position in geographic coordinates
*    **DIST_FROM_P93_BOW_SHOCK**: Distance from the P93 Bow Shock
*    **DIST_FROM_MAGNETOPAUSE**: Distance from the [RS93](https://doi.org/10.1029/93JA02362) Magnetopause
*    **DIST_FROM_T95_NS**: Distance to the [Tsyganenko 1995](https://doi.org/10.1029/94JA03193) model Neutral Sheet
*    **L_VALUE**: Dipole L value
*    **INVAR_LAT**: Dipole Invariant Latitude
*    **MAGNETIC_STRENGTH**: Magnetic Field Strength
*    **Dynamic Pressure (nPa)**: Peak dynamic pressure of jet 
*    **Electron density**: 
*    **Vx, Vy, Vz**: Cartesian coordinates of ion velocity. Used in computing dynamic pressure

These are pulled from [orbit parameters](https://cdaweb.gsfc.nasa.gov/misc/NotesT.html#THA_OR_SSC) and [on-board moment data](https://cdaweb.gsfc.nasa.gov/misc/NotesT.html#THA_L2_MOM), using [ai.cdas](https://pypi.org/project/ai.cdas/) and the second using [pyspedas](https://github.com/spedas/pyspedas/tree/master/pyspedas/themis) respectively.
       

In [2]:
mms_events = pd.read_fwf('https://osf.io/download/qyvzp/', sep="\s+", comment = '#', skiprows=21, names = ['Spacecraft', 'Start', 'Max', 'End'])
mms_events['Max']= pd.to_datetime(mms_events['Max']) # cast to datetime
mms_events['Start'] = pd.to_datetime(mms_events['Start'], format='%Y-%m-%d/%H:%M:%S')
mms_events['End'] = pd.to_datetime(mms_events['End'], format='%Y-%m-%d/%H:%M:%S')
mms_events = mms_events.set_index('Max')
mms_events

,Spacecraft,Start,End
Max,,,
2015-10-12 06:26:52,1,2015-10-12 06:26:27,2015-10-12 06:27:01
2015-10-12 06:34:00,1,2015-10-12 06:33:57,2015-10-12 06:34:03
2015-10-13 05:42:31,1,2015-10-13 05:42:09,2015-10-13 05:42:38
2015-10-13 05:43:03,1,2015-10-13 05:42:47,2015-10-13 05:43:16
2015-10-13 05:43:34,1,2015-10-13 05:43:25,2015-10-13 05:43:52
...,...,...,...
2017-02-05 07:18:50,4,2017-02-05 07:18:24,2017-02-05 07:19:41
2017-02-05 08:27:46,4,2017-02-05 08:27:25,2017-02-05 08:28:29
2017-02-05 09:15:36,4,2017-02-05 09:15:26,2017-02-05 09:15:40


In [3]:
mms_events = mms_events.head()

In [4]:
mms_events['SM_LAT']=""
event = 0
    # Load the DataFrame from the pickle file at the beginning
try:
   mms_events = pd.read_pickle('mms_event_list_with_coords.pkl')
   print("Successfully loaded data from pickle file.")
except FileNotFoundError:
   print("Pickle file not found. Let's start from the beginning...")
for epoch in mms_events.index:  # Iterate over timestamps directly
    event = event+1
    start = mms_events.loc[epoch, 'Start']
    end = mms_events.loc[epoch, 'End']
    try:
        print(f"Processing epoch: {epoch}")
        # Retrieve spacecraft data if not already present
        # if pd.isna(mms_events.loc[epoch, 'SM_LAT']):
        if(mms_events.loc[epoch, 'Vz']==""): #if we haven't run this event yet, let's run it:
            # Retrieve spacecraft data
            spacecraft_id = mms_events.loc[epoch, 'Ref Spacecraft']
            data_or_ssc = cdas.get_data(
                'sp_phys',
                f'TH{spacecraft_id}_OR_SSC',
                epoch, epoch + datetime.timedelta(minutes=15), # for the orbital parameters, get as close to epoch as possible - first non-NaN values!
                ['SM_LAT', 'SM_LON', 'XYZ_SM', 'XYZ_GEO', 'BOW_SHOCK', 'MAG_PAUSE', 'DNEUTS', 'L_VALUE', 'INVAR_LAT', 'MAG_STRTH', 'NorthBtrace_GM_LAT','NorthBtrace_GM_LON','NorthBtrace_GM_ARCLEN','SouthBtrace_GM_LAT','SouthBtrace_GM_LON','SouthBtrace_GM_ARCLEN']#, 'NorthBTrace_GEO_ARCLEN']#'dNeutS']#, 'BOW_SHOCK', 'MAG_PAUSE']
            )

            print("Successfully pulled orbit data for epoch.")

            # # Assign data to DataFrame

            print('Adding orbit coordinates to dataframe...')
            for parameter in ['SM_LAT', 'SM_LON']:#, 'DIST_FROM_T95_NS', 'DIST_FROM_MAGNETOPAUSE']:
                mms_events.loc[epoch, parameter] = data_or_ssc[parameter][0]

            for param in ["X", "Y", "Z"]:
                # print("Loading " + param)
                mms_events.loc[epoch, 'SM_' + param] = data_or_ssc[param][0]

            for param in ["X_1", "Y_1", "Z_1"]:
                # print("Loading " + param)
                mms_events.loc[epoch, 'GEO_' + param] = data_or_ssc[param][0]

            for parameter in ['DIST_FROM_P93_BOW_SHOCK', 'DIST_FROM_MAGNETOPAUSE', 'DIST_FROM_T95_NS', 'L_VALUE', 'INVAR_LAT', 'MAGNETIC_STRENGTH', 
                              # 'NORTHBTRACE_GM_LAT', 'NORTHBTRACE_GM_LON', 'NORTHBTRACE_GM_ARCLEN', 'SOUTHBTRACE_GM_LAT', 'SOUTHBTRACE_GM_LON', 'SOUTHBTRACE_GM_ARCLEN'
                             ]:
                foo = data_or_ssc[parameter]
                bar = foo[~pd.isnull(foo)]
                # mms_events.loc[epoch, parameter] = data_or_ssc[parameter][0]     
                mms_events.loc[epoch, parameter] = bar[0]     

            print('Pulling moment data...')


            mom_vars = pyspedas.mms.mom(probe=[spacecraft_id.lower()], trange=[start.timestamp(), end.timestamp()],
            # mom_vars = pyspedas.mms.mom(probe=[spacecraft_id.lower()], trange=[epoch.timestamp()-5*60, epoch.timestamp()+5*60],
                                   notplot=True,no_update=False,time_clip=True)



            thx_density = mom_vars['th'+spacecraft_id.lower()+'_peem_density']
            thx_density_time = thx_density['x']
            thx_density_data = thx_density['y']

            #calculate dynamic pressure
            thx_peim_velocity = mom_vars['th'+spacecraft_id.lower()+'_peim_velocity_gse']
            thx_velocity_data = thx_peim_velocity['y']

            thx_velocity_Vx = thx_velocity_data[:,0]
            thx_velocity_Vy = thx_velocity_data[:,1]
            thx_velocity_Vz = thx_velocity_data[:,2]

            print("Computing dynamic pressure for event ", event, " out of ", len(mms_events), ".")
            thx_pdy = thx_density_data*(thx_velocity_Vx**2+thx_velocity_Vy**2+thx_velocity_Vz**2)*1.67*(10**(-6))*0.5 # should return nPa

            dyp = thx_pdy[~pd.isnull(thx_pdy)] # dynamic pressure vector without NaNs

            mms_events.loc[epoch, 'Dynamic Pressure (nPa)'] = dyp.max()
            mms_events.loc[epoch, 'Electron density'] = thx_density_data[np.argmax(dyp)]        
            mms_events.loc[epoch, 'Vx'] = thx_velocity_Vx[np.argmax(dyp)]
            mms_events.loc[epoch, 'Vy'] = thx_velocity_Vy[np.argmax(dyp)]
            mms_events.loc[epoch, 'Vz'] = thx_velocity_Vz[np.argmax(dyp)]


            # Save the DataFrame after processing each epoch
            print("Saving data...")
            mms_events.to_csv('mms_event_list_with_coords.csv', encoding='utf-8')
            mms_events.to_pickle('mms_event_list_with_coords.pkl')
        else: 
            print("Got it!", event, " is already calculated. Onto the next epoch...")
    except Exception as e:
        print(f"Error occurred: {e}")
        continue

# mms_events.to_csv('mms_event_list_with_coords.csv', encoding='utf-8')
# mms_events.to_pickle('mms_event_list_with_coords.pkl')
        
# data_or_ssc.keys()
# mms_events


02-Jul-24 20:58:24: /tmp/ipykernel_890/869911564.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mms_events['SM_LAT']=""



Pickle file not found. Let's start from the beginning...
Processing epoch: 2015-10-12 06:26:52
Error occurred: 'Vz'
Processing epoch: 2015-10-12 06:34:00
Error occurred: 'Vz'
Processing epoch: 2015-10-13 05:42:31
Error occurred: 'Vz'
Processing epoch: 2015-10-13 05:43:03
Error occurred: 'Vz'
Processing epoch: 2015-10-13 05:43:34
Error occurred: 'Vz'


In [5]:
# And finally, we can open the recoverable files:

with open('themis_event_list_with_coords.pkl', 'rb') as f:
    mms_events = pickle.load(f)
mms_events

,Jet Number,Ref Spacecraft,Start,End,SM_LAT,SM_LON,SM_X,SM_Y,SM_Z,GEO_X_1,...,DIST_FROM_MAGNETOPAUSE,DIST_FROM_T95_NS,L_VALUE,INVAR_LAT,MAGNETIC_STRENGTH,Dynamic Pressure (nPa),Electron density,Vx,Vy,Vz
Max,,,,,,,,,,,,,,,,,,,,,
2012-12-16 23:56:43,2,A,2012-12-16 23:56:31,2012-12-16 23:57:07,-3.26014,19.5236,10.8744,3.85587,-0.657211,-10.60290,...,0.549921,-1.000000e+31,11.5940,72.9212,-1.000000e+31,2.894031,0.278323,-74.203164,41.282661,5.197419
2012-12-17 00:00:14,3,A,2012-12-17 00:00:10,2012-12-17 00:01:26,-3.43351,19.7288,10.8709,3.89850,-0.692903,-10.67800,...,0.561819,-1.000000e+31,11.6112,72.9343,-1.000000e+31,1.882750,10.259104,-35.856940,129.441884,-112.543848
2012-12-17 00:06:13,4,A,2012-12-17 00:06:03,2012-12-17 00:06:37,-3.69346,20.0407,10.8639,3.96289,-0.746493,-10.78580,...,0.578608,-1.000000e+31,11.6365,72.9534,-1.000000e+31,1.882750,10.259104,-35.856940,129.441884,-112.543848
2012-12-17 00:08:48,5,A,2012-12-17 00:08:35,2012-12-17 00:09:34,-3.78059,20.1425,10.8614,3.98382,-0.764470,-10.82060,...,0.583947,-1.000000e+31,11.6448,72.9596,-1.000000e+31,1.882750,10.259104,-35.856940,129.441884,-112.543848
2012-12-28 11:51:23,6,A,2012-12-28 11:51:00,2012-12-28 11:51:28,-2.73318,11.2758,10.9408,2.18139,-0.532586,10.95490,...,0.218966,-1.000000e+31,11.1943,72.6095,-1.000000e+31,2.385924,50.471666,-222.940315,82.164582,12.665370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-11-15 13:38:29,233,E,2018-11-15 13:34:01,2018-11-15 13:39:21,13.6009,345.1800,12.3067,-3.25628,3.079980,9.07944,...,2.135690,-1.000000e+31,13.8642,74.4210,-1.000000e+31,1.812517,13.711293,-88.002298,-45.436452,42.100172
2018-11-15 14:11:07,234,E,2018-11-15 14:10:23,2018-11-15 14:11:50,14.0414,345.9240,12.1040,-3.03483,3.120870,7.72699,...,1.903860,-1.000000e+31,13.6676,74.3065,-1.000000e+31,1.812517,13.711293,-88.002298,-45.436452,42.100172
2018-11-15 14:33:21,235,E,2018-11-15 14:32:53,2018-11-15 14:34:59,14.2569,346.4340,11.9562,-2.88489,3.125190,6.78092,...,1.732510,-1.000000e+31,13.5094,74.2125,-1.000000e+31,1.812517,13.711293,-88.002298,-45.436452,42.100172
